# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hancheng,32.5186,112.3522,5.79,74,91,2.36,CN,1706655255
1,1,waitangi,-43.9535,-176.5597,17.68,97,100,2.68,NZ,1706655112
2,2,yellowknife,62.4560,-114.3525,-10.33,100,100,4.63,CA,1706655113
3,3,adamstown,-25.0660,-130.1015,23.98,80,100,14.23,PN,1706655109
4,4,labuan,5.2767,115.2417,27.20,88,20,2.68,MY,1706655498


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display
# Configure the map plot with smaller dimensions
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="OSM",
    frame_width=800,  
    frame_height=400,  
    color="City",
    size="Humidity",
    alpha=0.5
)

# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_df_nas = city_data_df[
                 (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) &
                 (city_data_df["Wind Speed"] < 4.5) &
                 (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
perfect_df = perfect_df_nas.dropna(how='any')

# Display sample data
perfect_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,inhambane,-23.8650,35.3833,24.02,83,0,3.86,MZ,1706655262
46,46,port elizabeth,-33.9180,25.5701,22.40,86,0,1.34,ZA,1706655156
98,98,barra de santo antonio,-9.4047,-35.5072,26.60,78,0,4.29,BR,1706655515
183,183,quelimane,-17.8786,36.8883,26.97,78,0,1.03,MZ,1706655530
296,296,lompoc,34.6391,-120.4579,23.21,59,0,2.57,US,1706655149
364,364,champoton,19.3500,-90.7167,21.87,56,0,4.15,MX,1706655564
391,391,vallenar,-28.5708,-70.7581,26.66,49,0,2.39,CL,1706655232


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:


# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =perfect_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df



,City,Country,Lat,Lng,Humidity,Hotel Name
38,inhambane,MZ,-23.8650,35.3833,83,
46,port elizabeth,ZA,-33.9180,25.5701,86,
98,barra de santo antonio,BR,-9.4047,-35.5072,78,
183,quelimane,MZ,-17.8786,36.8883,78,
296,lompoc,US,34.6391,-120.4579,59,
364,champoton,MX,19.3500,-90.7167,56,
391,vallenar,CL,-28.5708,-70.7581,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 50000
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "categories":"accommodation.hotel"
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
inhambane - nearest hotel: Hotel de Inhambane
port elizabeth - nearest hotel: Waterford Hotel
barra de santo antonio - nearest hotel: Resort Fazenda Fiore
quelimane - nearest hotel: Mabassa
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
champoton - nearest hotel: Paraíso Beach
vallenar - nearest hotel: Hotel del Marqués


,City,Country,Lat,Lng,Humidity,Hotel Name
38,inhambane,MZ,-23.8650,35.3833,83,Hotel de Inhambane
46,port elizabeth,ZA,-33.9180,25.5701,86,Waterford Hotel
98,barra de santo antonio,BR,-9.4047,-35.5072,78,Resort Fazenda Fiore
183,quelimane,MZ,-17.8786,36.8883,78,Mabassa
296,lompoc,US,34.6391,-120.4579,59,Embassy Suites by Hilton Lompoc Central Coast
364,champoton,MX,19.3500,-90.7167,56,Paraíso Beach
391,vallenar,CL,-28.5708,-70.7581,49,Hotel del Marqués


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display
# Configure the map plot with smaller dimensions
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="OSM",
    frame_width=800,  
    frame_height=400,  
    color="City",
    size="Humidity",
    alpha=0.5,
    hover_cols=["Country", "Hotel Name"],
    legend="right"
)

# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)